In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

In [14]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# model = load_model('mask_detection_model.h5')
model = load_model('model2-007.h5')

labels = ['Without Mask', 'With Mask']

colors = [(0, 0, 255), (0, 255, 0)]

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]

        face = cv2.resize(face, (150, 150))

        face = np.expand_dims(face, axis=0)
        face = face / 255.0

        pred = model.predict(face)[0]

        label = labels[pred.argmax()]
        score = pred[pred.argmax()]

        print(label, " ", score)
        color = colors[pred.argmax()]
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, label + ' ({:.2f}%)'.format(score*100), (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    cv2.imshow('Face Mask Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [59]:
cap.release()
cv2.destroyAllWindows()


In [67]:
import cv2
import numpy as np

net = cv2.dnn.readNet('yolov4.weights', 'yolov4.cfg')

labels = ['Without Mask', 'With Mask']

colors = [(0, 255, 225), (0, 255, 0)]

model = load_model('mask_detection_model.h5')

conf_thresh = 0.8

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()

    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)

    net.setInput(blob)
    detections = net.forward()

    for detection in detections:
        class_id = np.argmax(detection[5:])
        score = detection[class_id + 5]
        if score > conf_thresh:
            box = detection[:4] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
            (x, y, w, h) = box.astype('int')

            face = frame[y:y+h, x:x+w]

            face = cv2.resize(face, (150, 150))

            face = np.expand_dims(face, axis=0)
            face = face / 255.0

            pred = model.predict(face)[0]

            label = labels[pred.argmax()]
            score = pred[pred.argmax()]

            color = colors[pred.argmax()]
            print(x,y,w,h)
            cv2.rectangle(frame, (int(x/2), int(y/2)), (int(x + x*0.5), int(y + y*0.5)), color, 2)
            cv2.putText(frame, label + ' ({:.2f}%)'.format(score*100), (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    cv2.imshow('Face Mask Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


318 274 650 410
319 274 640 408
317 274 655 404
318 274 645 403
318 277 659 402
318 277 650 399
318 277 662 401
319 277 651 399
316 275 668 406
317 275 658 405
318 275 663 406
318 275 655 404
318 273 664 406
318 273 656 403
319 273 667 407
320 273 658 404
318 275 665 409
319 275 655 406
318 274 670 406
318 274 660 403
317 273 666 408
317 273 657 405
318 273 668 409
318 273 658 406
318 273 674 407
318 273 664 403
318 272 667 409
318 272 657 405
317 273 661 409
318 272 651 406
318 273 664 409
318 273 655 407
318 272 658 414
318 272 650 410
